In [55]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
import plotly.express as px
from collections import Counter
from nltk.util import ngrams
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to C:\Users\Husein
[nltk_data]     Ghadiali\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to C:\Users\Husein
[nltk_data]     Ghadiali\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to C:\Users\Husein
[nltk_data]     Ghadiali\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Husein Ghadiali\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [56]:
df = pd.read_csv('master_review_v2.csv')
df.head()

,Product_name,SKU,Price,Subject,Date,Review,Stars,like,dislike
0,Mansfield Summit White Elongated Chair Height ...,Model #5384CTK,209.0,"Good toilet, complete kit w/ soft-close seat","July 12, 2021",Install go mostly accord plan however bolts se...,5,13,2
1,Mansfield Summit White Elongated Chair Height ...,Model #5384CTK,209.0,Easy Install,"July 3, 2021",easy install Make sure someone help carry box ...,5,6,0
2,Mansfield Summit White Elongated Chair Height ...,Model #5384CTK,209.0,Hotel Flusher,"August 6, 2021",first thing want say youve ever stayed hotel u...,4,10,1
3,Mansfield Summit White Elongated Chair Height ...,Model #5384CTK,209.0,Gorgeous toilet,"July 17, 2021",get toilet think different get installed appar...,5,6,3
4,Mansfield Summit White Elongated Chair Height ...,Model #5384CTK,209.0,Great functioning toilet,"August 29, 2021",toilet toilet pretty much Good thing toilet po...,5,8,0


In [57]:
df.shape

(380, 9)

In [58]:
# Convert datatypes to best possiblle
df = df.convert_dtypes()
# Convert date to datetime
df['Date'] = pd.to_datetime(df['Date'])
# Verifying the data types
df.dtypes

Product_name    string[python]
SKU             string[python]
Price                    Int64
Subject         string[python]
Date            datetime64[ns]
Review          string[python]
Stars                    Int64
like                     Int64
dislike                  Int64
dtype: object

In [59]:
# 1. Find the duration of the dataset
min_date = pd.to_datetime(df['Date']).min()
max_date = pd.to_datetime(df['Date']).max()
duration = max_date - min_date
start_date = min_date.strftime('%m/%d/%Y')
end_date = max_date.strftime('%m/%d/%Y')
print(f"The duration of the dataset is {duration} (from {start_date} to {end_date}).")

The duration of the dataset is 1342 days 00:00:00 (from 08/25/2019 to 04/28/2023).


In [60]:
# 3. See the distribution of total stars
stars_dist = df['Stars'].value_counts()
print("The distribution of total stars is:")
print(stars_dist)

The distribution of total stars is:
Stars
5    161
4     77
1     67
3     42
2     33
Name: count, dtype: Int64


In [61]:
# 4. Check if all models have the same price and the total models and their distribution
price_check = df.groupby(['SKU', 'Price']).size().reset_index(name='count')
if price_check['count'].nunique() == 1:
    print("All models have the same price.")
else:
    print("Not all models have the same price.")
    price_dist = df.groupby('SKU')['Price'].value_counts()
    print("The distribution of prices for each model is:")
    print(price_dist)

Not all models have the same price.
The distribution of prices for each model is:
SKU             Price
Model #5130CTK  149      116
Model #5384CTK  209      111
Model #5385CTK  239       34
Model #5916CTK  209       83
Model #5997CTK  249       36
Name: count, dtype: int64


# Time period where most product were sold

In [63]:
# Find dates when most reviews were written which would show when the products were purchased most
review_dates = df['Date'].value_counts().reset_index()
review_dates.columns = ['Date', 'Count']
fig = px.histogram(review_dates, x='Date', y='Count', title='Review Dates')
fig.show()

# Top selling product

In [64]:
# Find the top selling product amongst all by SKU
top_selling = df['SKU'].value_counts().reset_index()
top_selling.columns = ['SKU', 'Count']
fig = px.bar(top_selling, x='SKU', y='Count', color='SKU', title='Top Selling Products by SKU')
fig.show()

# N Grams

In [65]:
def get_valid_bigrams(tokens):
    bigrams = []
    tagged_tokens = pos_tag(tokens)
    for i in range(len(tagged_tokens) - 1):
        if (tagged_tokens[i][1].startswith('JJ') and tagged_tokens[i + 1][1].startswith('NN')) or \
           (tagged_tokens[i][1].startswith('NN') and tagged_tokens[i + 1][1].startswith('NN')):
            bigrams.append(f'{tagged_tokens[i][0]} {tagged_tokens[i + 1][0]}')
    return bigrams

In [66]:
def get_valid_trigrams(tokens):
    trigrams = []
    tagged_tokens = pos_tag(tokens)
    for i in range(len(tagged_tokens) - 2):
        if (tagged_tokens[i][1].startswith('JJ') and tagged_tokens[i + 1][1].startswith('NN') and tagged_tokens[i + 2][1].startswith('NN')) or \
           (tagged_tokens[i][1].startswith('NN') and tagged_tokens[i + 1][1].startswith('NN') and tagged_tokens[i + 2][1].startswith('NN')):
            trigrams.append(f'{tagged_tokens[i][0]} {tagged_tokens[i + 1][0]} {tagged_tokens[i + 2][0]}')
    return trigrams

In [67]:
# Find phrases (bi-gram, tri-gram) in reviews with 4,5 stars (Top 10)
positive_reviews = df[df['Stars'].isin([4, 5])]['Review']
positive_bigrams = []
positive_trigrams = []
for review in positive_reviews:
    review = review.lower()
    tokens = review.split()
    positive_bigrams.extend(get_valid_bigrams(tokens))
    positive_trigrams.extend(get_valid_trigrams(tokens))
top_positive_bigrams = Counter(positive_bigrams).most_common(10)
top_positive_trigrams = Counter(positive_trigrams).most_common(10)
top_positive_phrases_df = pd.DataFrame(top_positive_bigrams + top_positive_trigrams, columns=['Phrase', 'Count'])
fig = px.bar(top_positive_phrases_df, x='Phrase', y='Count', title='Top Positive Phrases')
fig.show()

In [69]:
# Find phrases (bi-gram, tri-gram) in reviews with 1,2,3 stars (Top 10)
negative_reviews = df[df['Stars'].isin([1, 2, 3])]['Review']
negative_bigrams = []
negative_trigrams = []
for review in negative_reviews:
    review = review.lower()
    tokens = review.split()
    negative_bigrams.extend(get_valid_bigrams(tokens))
    negative_trigrams.extend(get_valid_trigrams(tokens))
top_negative_bigrams = Counter(negative_bigrams).most_common(10)
top_negative_trigrams = Counter(negative_trigrams).most_common(10)
top_negative_phrases_df = pd.DataFrame(top_negative_bigrams + top_negative_trigrams, columns=['Phrase', 'Count'])
fig = px.bar(top_negative_phrases_df, x='Phrase', y='Count', title='Top Negative Phrases')
fig.show()

# Most negative review with highest like count

In [71]:
# Find the most negative review with highest like counts
negative_review = df[df['Stars'].isin([1, 2, 3])].sort_values(by=['like'], ascending=False).iloc[0]
print(f"The most negative review with the highest like count is: {negative_review['Review']}")

The most negative review with the highest like count is: two home good get waste inside wall go typically clean bowl design water wont push itor clean interior bowl work worth


"We have two of these in our home and both are not very good at getting waste off the inside wall. After going I typically have to clean the bowl because the design is such that the water won't push it,or clean it off the interior bowl. More work than it's worth"
Stars - 2
like - 19
dislike - 2

# Most positive review with highest like count

In [72]:
# Find the most positive review with highest like counts
positive_review = df[df['Stars'].isin([4, 5])].sort_values(by=['like'], ascending=False).iloc[0]
print(f"The most positive review with the highest like count is: {positive_review['Review']}")

The most positive review with the highest like count is: Lets face Im old fat mean Im usually cranky use bathroom lot use spend hundred dollar every quarter water bill toilet might say flush away money Dont get wrong still spend hundred dollar every quarter water bill indeed spend few hundred Since instal comfortable efficient toilet much happy experience first thing morning eat fiber Im still regular Laugh get old fat cranky regular major victory


"Let's face it. I'm old and fat. Which means I'm usually cranky and use the bathroom. A lot. I used to spend hundreds of dollars every quarter on my water bill. Before this toilet, you might say I was "flushing away money". Don't get me wrong, I still spend hundreds of dollars every quarter on my water bill BUT I do indeed spend fewer of those hundreds than I did. Since installing this comfortable and more efficient toilet, I now have a much happier experience first thing in the morning (I eat fiber so I'm still regular! Laugh if you will but when you get old and fat and cranky, being regular is a major victory!)"
Stars - 5
like - 26
dislike - 0

# Identifying Competitors

In [73]:
def extract_proper_nouns(text):
    # Tokenize the text and assign part of speech tags
    tokens = nltk.word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    
    # Use named entity recognition to extract proper nouns
    tree = nltk.ne_chunk(pos_tags)
    proper_nouns = [leaf[0] for subtree in tree if isinstance(subtree, nltk.tree.Tree) for leaf in subtree.leaves()]
    
    return proper_nouns

In [74]:
# Define a list of phrases to search for
phrases = ['buy from', 'go with', 'instead of', 'rather than', 'alternative to', 'better than', 'prefer', 'recommend', 'vs', 'versus',
           'compare', 'comparison', 'compared', 'compares', 'differ', 'different', 'differences', 'difference', 'opinion', 'opinions']

# Create a boolean mask to find reviews that contain any of the phrases
mask = df['Review'].str.contains('|'.join(phrases), case=False)

# Filter the DataFrame to only include rows where the mask is True
competitor_reviews = df[mask]

In [75]:
# Extract the competitor names and corresponding reviews from the reviews
competitor_names = []
competitor_reviews_list = []
for review in competitor_reviews['Review']:
    for phrase in phrases:
        if phrase in review.lower():
            start_index = review.lower().index(phrase) + len(phrase)
            words_after_phrase = review[start_index:]
            proper_nouns = extract_proper_nouns(words_after_phrase)
            if proper_nouns:
                competitor_name = proper_nouns[0]
                competitor_names.append(competitor_name)
                competitor_reviews_list.append(review)

In [76]:
# Create a DataFrame with the competitor names and corresponding reviews
competitor_data = pd.DataFrame({'Competitor': competitor_names, 'Review': competitor_reviews_list})
print(competitor_data)


   Competitor                                             Review
0       Delta  husband purchase toilet elderly parent rush ma...
1   Mansfield  take couple trip back Lowes buy jumbo toilet r...
2      Bought  Sleek modern design compare decade old commode...
3       Allow  Made USA great product happy Bought 2 house 3 ...
4       Allow  Made USA great product happy Bought 2 house 3 ...
5      Common  positive seem flush well Bottom half solid Neg...
6      Common  positive seem flush well Bottom half solid Neg...
7      Kohler  tank bolt rubberize enough tank leak since get...
8      Kohler  tank bolt rubberize enough tank leak since get...
9    American  never write review anything anywhere disappoin...
10    Overall  Likes Mansfield customer service awesome call ...
11   Dislikes  Likes Mansfield customer service awesome call ...
12      Lowes  Let preface article much despise plumbing proj...
13      Lowes  Let preface article much despise plumbing proj...
14      Lowes  Let prefac

### Competitors identified - American Standard, Kohler and Champion